In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import numpy as np

BACKTESTING_MODULE_PATH = os.path.abspath('../../backtest')
sys.path.insert(1, BACKTESTING_MODULE_PATH)
from backtesting.backtesting import Backtest, Strategy

METHOD_MODULE_PATH = os.path.abspath('../..')
sys.path.insert(1, METHOD_MODULE_PATH)
import method.Ticker as Ticker
import method.SaleRules as _sr
import method.algofuncs as _af
path = os.getcwd()

Loading BokehJS ...

In [9]:
DATA_PATH = os.path.abspath('../../vn-stock-data/VNX/')
ticker_id = 'PNJ'
ticker_data = _af.get_pricing_by_path(DATA_PATH + '/' + ticker_id+'.csv', '2018-01-01')

In [10]:
class StockBreakOut(Strategy):
    def init(self):
        self.buy_price = 0
        self._index = 0
    def next(self):
        prices = self.data.Close
        lowPrices = self.data.Low
        highPrices = self.data.High
        if len(self.data.Volume) > 66:
            last_price = prices[-1]
            week_price = highPrices[-5:-1]
            max_high_price_week = np.max(week_price)
            if self.buy_price == 0 and Ticker.isCounterTrendV3b(lowPrices, max_high_price_week):
                self.buy_price = last_price
                self.buy()
            if self.buy_price != 0 and (_sr.takeProfitByPercent(5, last_price, self.buy_price) or _sr.shouldCutLossByPercent(8, last_price, self.buy_price)):
                self.position.close()
                self.buy_price = 0

In [11]:
bt = Backtest(ticker_data, StockBreakOut, commission=.005, exclusive_orders=False)
stats = bt.run()

In [12]:
stats

Start                     2018-01-02 00:00:00
End                       2020-12-10 00:00:00
Duration                   1073 days 00:00:00
Exposure Time [%]                     42.1769
Equity Final [$]                      7212.65
Equity Peak [$]                       11048.2
Return [%]                           -27.8735
Buy & Hold Return [%]                 16.2704
Return (Ann.) [%]                    -10.5981
Volatility (Ann.) [%]                 18.1243
Sharpe Ratio                                0
Sortino Ratio                               0
Calmar Ratio                                0
Max. Drawdown [%]                    -47.1955
Avg. Drawdown [%]                    -24.7014
Max. Drawdown Duration      941 days 00:00:00
Avg. Drawdown Duration      478 days 00:00:00
# Trades                                   18
Win Rate [%]                          61.1111
Best Trade [%]                        10.2487
Worst Trade [%]                      -20.6651
Avg. Trade [%]                    

In [13]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,128,71,73,77.928705,80.3607,311.295360,0.031208,2018-04-20,2018-04-24,4 days
1,128,77,80,80.290254,86.0470,736.863488,0.071699,2018-05-03,2018-05-08,5 days
2,125,85,95,87.707154,76.1679,-1442.406750,-0.131566,2018-05-15,2018-05-29,14 days
3,123,96,101,78.014431,84.6833,820.270826,0.085482,2018-05-30,2018-06-06,7 days
4,123,108,111,84.391759,76.9154,-919.592218,-0.088591,2018-06-15,2018-06-20,5 days
5,132,113,118,71.626651,62.8026,-1164.774798,-0.123195,2018-06-22,2018-06-29,7 days
6,121,167,172,68.829636,72.4110,433.345044,0.052032,2018-09-07,2018-09-14,7 days
7,119,194,203,73.490022,65.6336,-934.914218,-0.106905,2018-10-16,2018-10-29,13 days
8,121,204,206,64.599490,68.9152,522.200850,0.066807,2018-10-30,2018-11-01,2 days
9,127,252,272,65.799360,70.1486,552.353480,0.066099,2019-01-08,2019-02-12,35 days


In [19]:
bt.plot()